In [1]:
%matplotlib inline 
exec(open("./stability.py").read()) 

# Stability analysis of the ENSO oscillator problem

Eingenvalue stability analysis is useful, at times more useful than the methods used for stability analysis so far, because we are not restricted to thinking about the problem in the limit $\Delta t \rightarrow 0$, which is something we cannot afford. Instead, we will consider a finite $\Delta t$, which we must choose for our numerical scheme.

The problem you were set with Jin's paper is governed by two coupled ODEs:

$$\frac{dT}{dt} = RT + \gamma h - \epsilon (h+bT)^3 + \gamma\xi \qquad$$
$$\frac{dh}{dt} = -rh - \alpha bT - \alpha\xi \qquad  ~~~~~~~~~~~~  (1)     $$ 

To understand the stabiliy of this sytem, we consider first the **analytical solutions** for the (simplest) linear case: the coupling coefficient $\mu$ is a constant between 0-1; no non-linearity ($\epsilon=0$), nor wind forcing ($\xi=0$). 

$$\frac{dT}{dt} = RT + \gamma h \qquad ~~~~~~~~~~~~~~~~~~$$
$$\frac{dh}{dt} = -rh - \alpha bT \qquad ~~~~~~~~~(2)$$ 


With these simplifications the system in (1) can be thought as a general problem in which, for a vector $\pmb{u}$, we have:
$$\frac{d\pmb{u}}{dt} = M\pmb{u} \qquad $$
In our specific case, $\pmb{u}$ has dimension 2 and the tuple $(T,h)$ are its elements, so that:

$$\frac{d}{dt} \begin{bmatrix}
           T \\
           h \\
         \end{bmatrix} = M \begin{bmatrix}
           T \\
           h \\
         \end{bmatrix} \qquad $$

where:
$$ M  = \begin{bmatrix}
            R  ~~~~~~~ \gamma  \\
            -\alpha b ~~ -r \\
         \end{bmatrix} \qquad $$


In general, the matrix M will not be diagonal, so that the components of the linearised system above will be coupled. One way to analyse the stability behaviour of this system is to use eigenvalue analysis (see the other notebook). Here we shall undertake an analytical approach.

Any variable $\phi(x,t)$ (thus dependent on space and time)  can be represented as a function of *wavenumber* $k$ and *frequency* $\omega$, for instance via a Fourier transformation:

$$ \phi(x,t) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} \tilde{\phi}(\kappa, \omega) e^{i(\omega t + \kappa x)} ~d \kappa ~d\omega $$

In our current problem (ENSO oscillator) we only have time dependence. Let us therefore use a solution of the type: $\phi(t) = \tilde{\phi(\omega)} e^{i\omega \tau \Delta t}$, where time $t=\tau \Delta t$. This means that any time $t$ in the future can be reached in a number ($\tau$) of time steps $\Delta t$. If, once we have discretised in time, we aim to step trom $\tau=n$ to $\tau=n+1$, all terms in our equations will contain a common exponential of the type: $e^{i\omega n \Delta t}$, which will cancel out, and only terms of type $e^{i\omega \Delta t}$, expressing the change in a single time step, will remain.

Therefore, from now on, to simplify our notation, we shall use this very useful shorthand in time:

$$\psi^1=e^{i\omega \Delta t}$$ 

A similar notation is applied in space, useful for future analyses, albeit not needed here!
$$\psi_1=e^{i\kappa \Delta x}$$
$$\psi_{-1}=e^{-i\kappa \Delta x}$$

Further, we must remember that $\omega=\omega_r+i\omega_i$ is a complex number, so that: $e^{i\omega t}=e^{i(\omega_r+i\omega_i)t}=e^{- \omega_i t} e^{i \omega_r t}$.

**It is therefore the imaginary term, $\omega_i$, that determines whether or not our solution grows in time ($\omega_i \gt 0$) or decays in time ($\omega_i \lt 0$)**. We use the symbol $\lambda$ to express this change of amplitude in a single time step:

$$\lambda = e^{- \omega_i \Delta t}$$

If we apply this framework to the the analysis of the stability of equation 1, after discretising in time with the **forward in time (Euler)** time scheme, and using all the notation introduced above, we end up with this system:

$$ (\psi^1 -1 -R \Delta t) \hat{T} ~~ - \gamma \Delta t \hat{h} = 0 \qquad     \\
   ~~~~~~(\psi^1 -1 +r \Delta t) \hat{h} ~~ + \alpha b \Delta t \hat{T} = 0 \qquad (3)$$

or, in matrix form:

$$\begin{bmatrix}
           (\psi^1 -1 -R \Delta t) ~~~~~~~~~~~~~~~ - \gamma \Delta t \\
             \alpha b \Delta t     ~~~~~~~~~~~~~~~~~~~  (\psi^1 -1 +r \Delta t)\\
         \end{bmatrix}  \begin{bmatrix}
           \hat{T} \\
           \hat{h} \\
         \end{bmatrix}
         = 0
         \qquad (4)$$


The only non-trivial solution to equation 4 will be found when the determinant of the matrix is zero. After a bit of algebra we end up with:

$$ {\psi^1}^2 + ((r-R) \Delta t -2)\psi^1 + 1 - (r-R) \Delta t  -Rr {\Delta t}^2  +\alpha\gamma b {\Delta t}^2=0 $$ 

Solving for ${\psi^1}$ and after a bit of simplification we find:

$$ {\psi^1} =  \frac {2 - A \pm  \sqrt{A^2  - 4C}}{2}$$

where $A= (r-R) \Delta t$ and $C= (-Rr +\alpha \gamma b) {\Delta t}^2$

This expression simplifies greatly for the neutral value of $\mu = \frac {2}{3}$, for which $A=0$, and:

$${\psi^1}= 1 \pm \Delta t \sqrt{-(\alpha\gamma b -Rr)} $$

Splitting the imaginary and real parts of this equation (remember that $e^{i \omega_r \Delta t} = \cos \omega_r \Delta t + i \sin \omega_r \Delta t$):

$$\lambda \cos \omega_r \Delta t = 1 \\
  \lambda \sin \omega_r \Delta t = \pm \Delta t \sqrt{\alpha\gamma b -Rr} \qquad (5)$$

Taking the squares of both equations and summing tells us that $\lambda^2 = 1+ ({\alpha\gamma b -Rr})\Delta t^2 $, so that $\lambda\gt 1$, which means that the Euler time scheme is unconditionally unstable for this problem.


## Stability analysis of the Euler scheme using SimPy
It is possible to repeat all of the above with a little help from SimPy. Careful that the parser has trouble with R and r, at times displaying it all as $R^2$, but it does not matter in the case of $\mu=2/3$.

In [2]:
from IPython.display import display, Latex, Math
import sympy
from sympy import *
from sympy.matrices import Matrix
init_printing(use_latex='mathjax')
# EULER SCHEME
Psi = sympy.symbols('Psi')             # these are the symbols to solve for
R, Dt, gamma, alpha, b, r = sympy.var('R, Dt, gamma, alpha, b, r')  # 
#M=Matrix(([[Psi-1-R*Dt,-gamma*Dt],[alpha*b*Dt,Psi-1+r*Dt]]))
M=Matrix(([[Psi-1-R*Dt,-gamma*Dt],[alpha*b*Dt,Psi-1+R*Dt]])) # special case of mu=2/3 so R=r

display(Math("{Matrix}\ M:\ " + latex(M)))

equation=M.det()
Solution=solve(equation,Psi)
#pprint(Solution)
Psi_plus=Solution[1]
Psi_minus=Solution[0]

print(" ")
display(Math("\Psi \ positive\ solution:\ " + latex(Psi_plus)))
display(Math("\Psi \ negative\ solution:\ " + latex(Psi_minus)))

print(" ")
mu=2./3.
b_0=2.5
b_val=b_0/mu
gamma_val=0.75
alpha_val=0.25
r_val=0.125
c_val=1.0
R_val=gamma_val*b_val - c_val

Psi_solution_plus  = Psi_plus.subs([(R,R_val), (r,r_val)])
Psi_solution_minus = Psi_minus.subs([(R,R_val), (r,r_val)])

lambda_squared = 1 + (alpha*gamma*b -R*r)*Dt**2

display(Math("Analytic\ expression\ for\ \lambda^2 :\  " + latex(lambda_squared)))
lambda_square_solution = lambda_squared.subs([(R,R_val), (r,r_val),(alpha,alpha_val), (gamma, gamma_val), (b,b_val)])
print(" ")
display(Math("Value\ of\ \lambda^2 \ for\ Euler:\  " + latex(lambda_square_solution)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Stability analysis of the modified Euler scheme (Matsuno scheme)
Use the forward solution for the first line in equation 1 (for $\frac{dT}{dt}$) and substitute into the second line of equation 1 (for $\frac{dh}{dt}$). This gives us a scheme with the flavour of an implicit-in-time scheme (remember the Matsuno scheme?). Try now to substitute the relevant $\psi^1$ terms in the analysis above, and see whether this technique leads to a more stable scheme in time.

## Stability analysis of the Runge-Kutta scheme
Try now to substitute the 1st and 2nd order RK scheme in the analysis above.